In [47]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

def fourier_decomp(layers_params):
    """
    Returns the frequencies and coefficient of our quantum model, specified by layers_params

    Args:
    layers_params: list(list(list(float))). Specifies the number of basic entangling layers and their
    parameters as explained in the statement of the problem.

    Returns: list([float,float,float]). A list three-element list. The first element of each list is the frequency. The second
    element is the real part of the coefficient associated with that frequency in the Fourier decomposition. The third element
    is the imaginary part of such coefficient.
    """

    dev = qml.device("default.qubit", wires=4)
        
    @qml.qnode(dev)
    def circuit(layers_params, input_data):
        """
        This function is the quantum circuit made of alternating entangling layers and rotations representing our quantum model
        """
        # Put your code here #
        # Return a single expectation value!
        for section in layers_params:
            qml.BasicEntanglerLayers(section, wires=[0,1,2,3])
            if section == layers_params[-1]:
                continue
            else:
                qml.RX(input_data, wires=[0])
                qml.RX(input_data, wires=[1])
                qml.RX(input_data, wires=[2])
                qml.RX(input_data, wires=[3])
        
        return qml.expval(qml.PauliZ([0]))

    # Use the Fourier module to obtain the coefficients and frequencies. Then return the required list of lists.
    print(layers_params)
    degree = 4*(len(layers_params)-1)
    partial_circuit = functools.partial(circuit, layers_params)
    coeffs = qml.fourier.coefficients(partial_circuit, 1, degree, lowpass_filter=True)
    print(coeffs)
    return [[i-degree, np.real(coeffs[i-degree]), np.imag(coeffs[i-degree])] for i in range(len(coeffs))]
    
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = fourier_decomp(ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated Fourier spectrum isn't quite right."


test_cases = [['[[[2,2,2,2],[1,2,1,1]],[[3,4,5,6]]]', '[[-4.0, -2.4671622769447922e-17, -1.2335811384723961e-17], [-3.0, -0.03395647263976357, 0.010208410500915437], [-2.0, 2.8360500437920326e-17, 1.850371707708594e-17], [-1.0, 0.11762992558035439, -0.13619443127813127], [0.0, 8.018277400070575e-17, 0.0], [1.0, 0.11762992558035439, 0.13619443127813124], [2.0, 3.700743415417188e-17, -1.850371707708594e-17], [3.0, -0.03395647263976357, -0.010208410500915437],[4.0, -3.688877668472405e-18, 1.850371707708594e-17]]'], ['[[[2,2,2,2]],[[3,4,5,6]]]', '[[-4.0, 1.2335811384723961e-17, 3.700743415417188e-17],  [-3.0, 0.022482345076620468, -0.07855141721016852], [-2.0, -1.2335811384723961e-17, -6.536793459209221e-17], [-1.0, -0.13243693333822854, 0.17097830099559677], [0.0, -2.4671622769447922e-17, 0.0], [1.0, -0.13243693333822854, -0.17097830099559677], [2.0, -2.4671622769447922e-17, 7.401486830834377e-17], [3.0, 0.022482345076620468, 0.07855141721016852], [4.0, -1.2335811384723961e-17, -3.331855648569948e-17]]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[[2,2,2,2],[1,2,1,1]],[[3,4,5,6]]]'...
[[[2, 2, 2, 2], [1, 2, 1, 1]], [[3, 4, 5, 6]]]
[-1.04083409e-17+0.00000000e+00j  1.17629926e-01+1.36194431e-01j
  6.93889390e-18-2.08166817e-17j -3.39564726e-02-1.02084105e-02j
  7.67592783e-17+3.71854204e-18j  6.93889390e-17-4.16333634e-17j
 -3.39564726e-02+1.02084105e-02j -7.37033925e-18+5.17926092e-17j
  1.17629926e-01-1.36194431e-01j]
Correct!
Running test case 1 with input '[[[2,2,2,2]],[[3,4,5,6]]]'...
[[[2, 2, 2, 2]], [[3, 4, 5, 6]]]
[ 3.46944695e-18+0.00000000e+00j -1.32436933e-01-1.70978301e-01j
 -6.93889390e-18+4.85722573e-17j  2.24823451e-02+7.85514172e-02j
 -5.50129611e-17+2.12481271e-17j  0.00000000e+00+1.38777878e-17j
  2.24823451e-02-7.85514172e-02j  4.11351733e-17-6.28814905e-17j
 -1.32436933e-01+1.70978301e-01j]
Correct!
